In [1]:
import pandas as pd
import numpy as np

In [2]:
data_train = pd.read_csv("C:/datasets/research_task/data_train.csv")
data_train = data_train.replace('?', np.nan)
data_v = data_train[data_train['target'] > 800].index
data_train = data_train.drop(data_v, axis=0)
data_train = data_train.drop(["id", "reopen_count"], axis=1)
data_train = data_train.dropna(axis = 1, thresh=1000)
trash_data = ["active", "made_sla"]
data_train = data_train.drop(trash_data, axis=1)

In [3]:
id_cols = ["caller_id", "sys_created_by", "sys_updated_by", "location", "assigned_to", "assignment_group",
           "resolved_by", "opened_by", "category", "subcategory", "u_symptom"]
data_cols = ["opened_at", "sys_created_at", "sys_updated_at"]

n_id_cols = ["caller_id", "sys_created_by", "sys_updated_by", "assigned_to", "assignment_group",
           "resolved_by", "opened_by"]

In [4]:
data_train["data_opened_at"] = data_train["opened_at"].astype("datetime64").dt.month.astype("str") + "-m"
data_train["data_sys_created_at"] = data_train["sys_created_at"].astype("datetime64").dt.month.astype("str") + "-m"
data_train["data_sys_updated_at"] = data_train["sys_updated_at"].astype("datetime64").dt.month.astype("str") + "-m"
data_train["data_created_updated_diff"] = (data_train["sys_updated_at"].astype("datetime64") - 
    data_train["sys_created_at"].astype("datetime64")).astype("timedelta64[h]")
data_train["data_opened_created_diff"] = (data_train["sys_created_at"].astype("datetime64") - 
    data_train["opened_at"].astype("datetime64")).astype("timedelta64[h]")

data_train = data_train.drop(data_cols, axis=1)

In [5]:
data_train["data_created_updated_diff"] = data_train["data_created_updated_diff"].replace(np.nan, 
                                                                                          data_train["data_created_updated_diff"].mean())
data_train["data_opened_created_diff"] = data_train["data_opened_created_diff"].replace(np.nan, 
                                                                                          data_train["data_opened_created_diff"].mean())

In [6]:
#Min-Max нормировка
data_train["data_created_updated_diff"] = (data_train["data_created_updated_diff"] - data_train["data_created_updated_diff"].min()) / (data_train["data_created_updated_diff"].max() - data_train["data_created_updated_diff"].min())
data_train["data_opened_created_diff"] = (data_train["data_opened_created_diff"] - data_train["data_opened_created_diff"].min()) / (data_train["data_opened_created_diff"].max() - data_train["data_opened_created_diff"].min())
data_train["sys_mod_count"] = (data_train["sys_mod_count"] - data_train["sys_mod_count"].min()) / (data_train["sys_mod_count"].max() - data_train["sys_mod_count"].min())
data_train["reassignment_count"] = (data_train["reassignment_count"] - data_train["reassignment_count"].min()) / (data_train["reassignment_count"].max() - data_train["reassignment_count"].min())

In [7]:
cat_cols = [col for col in data_train.select_dtypes(include=['object']).columns]
num_cols = [col for col in data_train.select_dtypes(include=['int64', 'bool', 'float64']).columns]
f_num_cols = [col for col in num_cols if  (col != 'target')]
n_f_cols = [col for col in data_train.columns if  (col != 'target')]

In [8]:
data_prom = pd.read_csv("C:/datasets/research_task/data_prom.csv")

In [9]:
data_prom = data_prom.replace('?', np.nan)

In [10]:
data_prom["data_opened_at"] = data_prom["opened_at"].astype("datetime64").dt.month.astype("str") + "-m"
data_prom["data_sys_created_at"] = data_prom["sys_created_at"].astype("datetime64").dt.month.astype("str") + "-m"
data_prom["data_sys_updated_at"] = data_prom["sys_updated_at"].astype("datetime64").dt.month.astype("str") + "-m"
data_prom["data_created_updated_diff"] = (data_prom["sys_updated_at"].astype("datetime64") - 
    data_prom["sys_created_at"].astype("datetime64")).astype("timedelta64[h]")
data_prom["data_opened_created_diff"] = (data_prom["sys_created_at"].astype("datetime64") - 
    data_prom["opened_at"].astype("datetime64")).astype("timedelta64[h]")

In [11]:
data_prom["data_created_updated_diff"] = data_prom["data_created_updated_diff"].replace(np.nan, 
                                                                                          data_prom["data_created_updated_diff"].mean())
data_prom["data_opened_created_diff"] = data_prom["data_opened_created_diff"].replace(np.nan, 
                                                                                          data_prom["data_opened_created_diff"].mean())

In [12]:
#Min-Max нормировка
data_prom["data_created_updated_diff"] = (data_prom["data_created_updated_diff"] - data_prom["data_created_updated_diff"].min()) / (data_prom["data_created_updated_diff"].max() - data_prom["data_created_updated_diff"].min())
data_prom["data_opened_created_diff"] = (data_prom["data_opened_created_diff"] - data_prom["data_opened_created_diff"].min()) / (data_prom["data_opened_created_diff"].max() - data_prom["data_opened_created_diff"].min())
data_prom["sys_mod_count"] = (data_prom["sys_mod_count"] - data_prom["sys_mod_count"].min()) / (data_prom["sys_mod_count"].max() - data_prom["sys_mod_count"].min())
data_prom["reassignment_count"] = (data_prom["reassignment_count"] - data_prom["reassignment_count"].min()) / (data_prom["reassignment_count"].max() - data_prom["reassignment_count"].min())

In [13]:
data_prom = data_prom[n_f_cols]

In [14]:
for col in cat_cols:
    for f in pd.unique(data_prom[col]):
        if (f not in pd.unique(data_train[col])):
            data_prom = data_prom.replace({col:{f:np.nan}})
for col in cat_cols:
    for f in pd.unique(data_train[col]):
        if (f not in pd.unique(data_prom[col])):
            data_train = data_train.replace({col:{f:np.nan}})

In [15]:
data_prom = data_prom.replace(np.nan, '?')
data_train = data_train.replace(np.nan, '?')

In [16]:
dummy_features = pd.get_dummies(data_prom[cat_cols])
data_prom_d = pd.concat([data_prom[f_num_cols], dummy_features], axis=1)
data_prom_d = data_prom_d.astype('float')

In [21]:
dummy_features = pd.get_dummies(data_train[cat_cols])
data_train_d = pd.concat([data_train[num_cols], dummy_features], axis=1)
data_train_d = data_train_d.astype('float')
for col in data_train_d.columns:
    if (col not in data_prom_d.columns and col != 'target'):
        data_train_d = data_train_d.drop(col, axis=1)
f_cols = [col for col in data_train_d.columns if (col != 'target')]

In [22]:
data_train_d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10019 entries, 0 to 10571
Columns: 2857 entries, reassignment_count to data_sys_updated_at_9-m
dtypes: float64(2857)
memory usage: 218.5 MB


In [23]:
data_prom_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Columns: 2856 entries, reassignment_count to data_sys_updated_at_9-m
dtypes: float64(2856)
memory usage: 121.3 MB


In [24]:
import sklearn
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_train_d[f_cols], data_train_d['target'], 
                                                       train_size=0.95, 
                                                       random_state=42)

In [25]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import data
import random
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import Dataset
from torchvision import datasets

In [26]:
class MyDataset(Dataset):
    def __init__(self, samples, labels):
        self.labels = torch.from_numpy(labels.to_numpy()).type(torch.FloatTensor) 
        self.samples = torch.from_numpy(samples.to_numpy()).type(torch.FloatTensor) 

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        label = self.labels[idx]
        return sample, label

In [30]:
class PromDataset(Dataset):
    def __init__(self, samples, num):
        self.samples = torch.from_numpy(samples.to_numpy()).type(torch.FloatTensor)
        self.num = num

    def __len__(self):
        num = self.num
        return num

    def __getitem__(self, idx):
        sample = self.samples[idx]
        return sample

In [31]:
t_data = MyDataset(X_train, y_train)
v_data = MyDataset(X_test, y_test)
p_data = PromDataset(data_prom_d, 5565)

In [32]:
dataloaders = {'train': data.DataLoader(t_data, batch_size=1,
                                              shuffle=True, num_workers=0),
               'val': data.DataLoader(v_data, batch_size=1,
                                              shuffle=True, num_workers=0),
               'prom': data.DataLoader(p_data, batch_size=1,
                                              shuffle=False, num_workers=0),
               }
dataset_sizes = {'train': len(t_data),
                'val': len(v_data), 'val': len(p_data)}

In [33]:
use_gpu = torch.cuda.is_available()

In [34]:
from tqdm.autonotebook import tqdm, trange

import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time

In [35]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()
    best_model_wts = model.state_dict()
    best_loss = 20000
    losses = {'train': [], "val": []}
    pbar = trange(num_epochs, desc="Epoch:")
    for epoch in pbar:
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)
            else:
                model.eval()
            running_loss = 0.0
            for data in tqdm(dataloaders[phase], leave=False, desc=f"{phase} iter:"):
                inputs, labels = data
                if use_gpu:
                    inputs = inputs.cuda()
                    labels = labels.cuda()
                if phase=="train":
                    optimizer.zero_grad()
                if phase == "eval":
                    with torch.no_grad():
                        outputs = model(inputs)
                else:
                    outputs = model(inputs)
                loss = criterion(outputs, labels)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                running_loss += loss.item()
            epoch_loss = running_loss / dataset_sizes[phase]        
            losses[phase].append(epoch_loss)  
            pbar.set_description('{} Loss: {:.4f}'.format(
                                    phase, epoch_loss
                                ))
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = model.state_dict()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f}'.format(best_loss))
    model.load_state_dict(best_model_wts)
    return model, losses

In [36]:
class SimpleNet(nn.Module):
    def __init__(self, num):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(num, 1000)
        self.dropout1 = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(1000, 500)
        self.dropout2 = nn.Dropout(p=0.3)
        self.fc3 = nn.Linear(500, 300)
        self.fc4 = nn.Linear(300, 1)
        
    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = self.dropout1(F.sigmoid(self.fc2(x)))
        x = self.dropout2(F.sigmoid(self.fc3(x)))
        x = self.fc4(x)
        return x

In [58]:
def run(model):
    model.eval()
    outputs = []
    for data in dataloaders['prom']:
        inputs = data
        if use_gpu:
            inputs = inputs.cuda()    
        output = model(inputs)[0][0].tolist()
        outputs.append(output)
    return outputs

In [38]:
model = SimpleNet(len(f_cols))
if use_gpu:
    model = model.cuda()
loss_fn = nn.L1Loss()
optimizer_ft = optim.Adam(model.parameters(), lr=1e-3)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

In [39]:
model, losses = train_model(model, loss_fn, optimizer_ft, num_epochs=15)

Epoch::   0%|          | 0/15 [00:00<?, ?it/s]

train iter::   0%|          | 0/9518 [00:00<?, ?it/s]

C:\Users\savel\anaconda3\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
C:\Users\savel\anaconda3\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


val iter::   0%|          | 0/501 [00:00<?, ?it/s]

train iter::   0%|          | 0/9518 [00:00<?, ?it/s]

...

train iter::   0%|          | 0/9518 [00:00<?, ?it/s]

val iter::   0%|          | 0/501 [00:00<?, ?it/s]

train iter::   0%|          | 0/9518 [00:00<?, ?it/s]

val iter::   0%|          | 0/501 [00:00<?, ?it/s]

Training complete in 8m 38s
Best val Loss: 6.117569


In [72]:
outs = run(model)

C:\Users\savel\anaconda3\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [73]:
outs = pd.DataFrame(outs,columns=['Outputs'])

In [74]:
outs.to_csv("C:/datasets/research_task/outs.csv") 